### Input song attribute data

In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm

df = pd.read_csv("pure_genre_data.csv")

df.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,...,track_id,uri,track_href,analysis_url,duration_ms,time_signature,artist_name,artist_id,track_name,genre
0,0.612,0.807,10,-2.810,1,0.0336,0.04950,0.017700,0.101,0.398,...,2QjOHCTQ1Jl3zawyYOpxh6,spotify:track:2QjOHCTQ1Jl3zawyYOpxh6,https://api.spotify.com/v1/tracks/2QjOHCTQ1Jl3...,https://api.spotify.com/v1/audio-analysis/2QjO...,240400,4,The Neighbourhood,77SW9BnxLY8rJ0RciFqkHh,Sweater Weather,alt-rock
1,0.575,0.568,9,-5.509,0,0.0300,0.04840,0.000417,0.286,0.370,...,2K7xn816oNHJZ0aVqdQsha,spotify:track:2K7xn816oNHJZ0aVqdQsha,https://api.spotify.com/v1/tracks/2K7xn816oNHJ...,https://api.spotify.com/v1/audio-analysis/2K7x...,206280,4,The Neighbourhood,77SW9BnxLY8rJ0RciFqkHh,Softcore,alt-rock
2,0.588,0.521,10,-9.461,1,0.0329,0.06780,0.149000,0.123,0.337,...,5E30LdtzQTGqRvNd7l6kG5,spotify:track:5E30LdtzQTGqRvNd7l6kG5,https://api.spotify.com/v1/tracks/5E30LdtzQTGq...,https://api.spotify.com/v1/audio-analysis/5E30...,260173,4,The Neighbourhood,77SW9BnxLY8rJ0RciFqkHh,Daddy Issues,alt-rock
3,0.471,0.924,8,-3.906,1,0.0586,0.00301,0.000000,0.313,0.725,...,2iUmqdfGZcHIhS3b9E9EWq,spotify:track:2iUmqdfGZcHIhS3b9E9EWq,https://api.spotify.com/v1/tracks/2iUmqdfGZcHI...,https://api.spotify.com/v1/audio-analysis/2iUm...,177280,4,Neon Trees,0RpddSzUHfncUWNJXKOsjy,Everybody Talks,alt-rock
4,0.551,0.881,7,-6.099,0,0.0542,0.18600,0.079100,0.152,0.387,...,7zwn1eykZtZ5LODrf7c0tS,spotify:track:7zwn1eykZtZ5LODrf7c0tS,https://api.spotify.com/v1/tracks/7zwn1eykZtZ5...,https://api.spotify.com/v1/audio-analysis/7zwn...,153000,4,The Neighbourhood,77SW9BnxLY8rJ0RciFqkHh,You Get Me So High,alt-rock


### Add in lyric data from lyricgenius API

In [2]:
# Import necessary libraries
!pip install lyricsgenius
import lyricsgenius

     |████████████████████████████████| 59 kB 8.3 MB/s 
     |████████████████████████████████| 128 kB 37.8 MB/s 
You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.


In [3]:
# Gain access to the api
access_token = "me-MXmuCTyJWDZ1MmERAAyivlY61EvPSD7KX2lBynV60vCOlEheYc6HVfPGlQ0ru"
genius = lyricsgenius.Genius(access_token)

# And we'll remove the section headers
genius.remove_section_headers = True

In [4]:
# Define a function that returns the lyrics of a particular song
def grab_lyric(row):

    # Grab the lyrics
    lyr = genius.search_song(row['track_name'], row['artist_name'])

    try: 
        lyr = lyr.lyrics
    except:
        pass

    return lyr

In [5]:
# Split out the fields we need
int_df = df[['track_id', 'artist_name', 'track_name']]

In [6]:
# Split the dataset into groupings
num_groups = round(int_df.shape[0]/40, 0)
subDFs = np.array_split(int_df, num_groups)

# Create final data frame that we'll concatenate to
final = pd.DataFrame()

# Loop over the groupings and pull the lyric data
for partition in tqdm(subDFs):
    try:
        partition['lyric_raw'] = partition.apply(grab_lyric, axis = 1)
        final = final.append(partition)
    except:
        print('failure to load track lyrics')


Searching for "AM" by Nio Garcia...
Done.
Searching for "Pa Mí - Remix" by Dalex...
Done.
Searching for "Perfecta" by Reik...
Done.
Searching for "Los Tragos" by Reik...
Done.
Searching for "Otra vez (feat. J Balvin)" by Zion & Lennox...
Done.
Searching for "Si Tu Me Busca" by Anuel AA...
Done.
Searching for "Mi Niña" by Wisin...
Done.
Searching for "El Efecto" by Rauw Alejandro...
Done.
Searching for "La Nota" by Manuel Turizo...
Done.
Searching for "Dura" by Daddy Yankee...
Done.
Searching for "Fuiste Tú (feat. Gaby Moreno)" by Ricardo Arjona...
Done.
Searching for "Chambea" by Bad Bunny...
Done.
Searching for "Borro Cassette" by Maluma...
Done.
Searching for "Niña Bonita" by Chino & Nacho...
Done.
Searching for "China" by Anuel AA...
Done.
Searching for "Amanece" by Anuel AA...
Done.
Searching for "Ella Me Levanto" by Daddy Yankee...
Done.
Searching for "Te Boté" by Darell...
Done.
Searching for "Diosa" by Myke Towers...
 93%|█████████▎| 163/175 [1:13:52<05:35, 27.98s/it]Done.
Sear

In [13]:
# Join the lyric data back to the original data frame
df_w_lyrics = df.merge(final, how = 'left', on = 'track_id')

### Clean up the lyric data

In [14]:
# Various imports needed for cleaning process
import nltk
from nltk.corpus import stopwords
import string
import re
from nltk.tokenize import word_tokenize
nltk.download('punkt')
from nltk.stem.porter import PorterStemmer

# Configure stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

# Configure punctuation set
punct = set(string.punctuation)

# Configure the stemmer
stemmer = PorterStemmer()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [15]:
# Define a function that passes in a raw lyric and outputs a cleaned list of stems
def clean_lyrics(lyric):

    try:
        # Convert to all lowercase
        lyric_int_1 = lyric.lower()

        # Remove punctuation
        lyric_int_2 = ''.join([ch for ch in lyric_int_1 if ch not in punct])

        # Remove all newline characters
        lyric_int_3 = re.sub('\n', ' ', lyric_int_2)

        # Split long string into list of words (bundled characters)
        lyric_int_4 = word_tokenize(lyric_int_3)

        # Remove stopwords
        lyric_int_5 = [word for word in lyric_int_4 if word not in stop_words]

        # Stem the list of words
        lyric_int_6 = [stemmer.stem(word) for word in lyric_int_5]

        # Finally, remove first line (title); we do this by finding first instance 
        # of 'lyric' stem in list above and only pulling list directly after this index
        
        # We also remove the very last word since it appears that there's some sort of
        # embedding code attached to it, and the omission of one word shouldn't affect
        # the modelling.
        lyric_final = lyric_int_6[lyric_int_6.index('lyric')+1:-2]

        return lyric_final

    except:
        pass

In [16]:
# Apply the function above to the lyrics field
df_w_lyrics['lyric_clean'] = df_w_lyrics['lyric_raw'].apply(clean_lyrics)

In [18]:
df_w_lyrics.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,...,duration_ms,time_signature,artist_name_x,artist_id,track_name_x,genre,artist_name_y,track_name_y,lyric_raw,lyric_clean
0,0.612,0.807,10,-2.810,1,0.0336,0.04950,0.017700,0.101,0.398,...,240400,4,The Neighbourhood,77SW9BnxLY8rJ0RciFqkHh,Sweater Weather,alt-rock,NaN,NaN,NaN,None
1,0.575,0.568,9,-5.509,0,0.0300,0.04840,0.000417,0.286,0.370,...,206280,4,The Neighbourhood,77SW9BnxLY8rJ0RciFqkHh,Softcore,alt-rock,NaN,NaN,NaN,None
2,0.588,0.521,10,-9.461,1,0.0329,0.06780,0.149000,0.123,0.337,...,260173,4,The Neighbourhood,77SW9BnxLY8rJ0RciFqkHh,Daddy Issues,alt-rock,NaN,NaN,NaN,None
3,0.471,0.924,8,-3.906,1,0.0586,0.00301,0.000000,0.313,0.725,...,177280,4,Neon Trees,0RpddSzUHfncUWNJXKOsjy,Everybody Talks,alt-rock,NaN,NaN,NaN,None
4,0.551,0.881,7,-6.099,0,0.0542,0.18600,0.079100,0.152,0.387,...,153000,4,The Neighbourhood,77SW9BnxLY8rJ0RciFqkHh,You Get Me So High,alt-rock,NaN,NaN,NaN,None


In [17]:
song = 11

raw = df_w_lyrics.iloc[song, -2]
clean = df_w_lyrics.iloc[song, -1]

print(raw)
print(clean)

nan
None


### Write results

In [12]:
df_w_lyrics.to_csv('pure_genre_data_w_clean_lyrics.csv', index = False)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=659c715d-e2b5-478e-9116-4d32a5174810' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>